<a href="https://colab.research.google.com/github/MChujyo/SolarFlarePrediction/blob/main/vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pip install torch torchvision transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTConfig
from sklearn.model_selection import train_test_split

def tss_loss(outputs, targets):
    probabilities = torch.sigmoid(outputs)[:,1]
    true_positive = ((probabilities >= 0.5) & (targets == 1)).float().sum()
    false_negative = ((probabilities < 0.5) & (targets == 1)).float().sum()
    false_positive = ((probabilities >= 0.5) & (targets == 0)).float().sum()
    true_negative = ((probabilities < 0.5) & (targets == 0)).float().sum()

    tpr = true_positive / (true_positive + false_negative + 1e-6)
    fpr = false_positive / (false_positive + true_negative + 1e-6)

    tss = tpr - fpr
    loss = 1 - tss
    return loss.requires_grad_(True)  # 勾配を必要とするように設定


class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

images = np.load('/content/drive/MyDrive/2024/GeoSciAi/datas/train_mag.npy')
images = images.transpose(2,0,1)
labels = np.load('/content/drive/MyDrive/2024/GeoSciAi/datas/train_label.npy')
images = images.astype(np.float32)
labels = labels.astype(np.int64)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = CustomDataset(train_images, train_labels, transform=transform)
test_dataset = CustomDataset(test_images, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=15, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=15, shuffle=False)

config = ViTConfig.from_pretrained('google/vit-base-patch16-224', num_channels=1, num_labels=2)
model = ViTForImageClassification(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train(epoch):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).long()
        outputs = model(images).logits
        #loss = tss_loss(outputs, labels)
        loss=nn.CrossEntropyLoss()(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {loss.item()}")

for epoch in range(10):
    train(epoch)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 0, Step 0, Loss: 0.7144231200218201
Epoch 0, Step 10, Loss: 0.6027016043663025


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def tss_score(true_labels, predictions):
    cm = confusion_matrix(true_labels, predictions)  # 混同行列を取得
    TN, FP, FN, TP = cm.ravel()  # 混同行列から値を取得
    TPR = TP / (TP + FN + 1e-6)  # 真陽性率
    FPR = FP / (FP + TN + 1e-6)  # 偽陽性率
    return TPR - FPR  # TSS計算


def evaluate(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images).logits
            predicted_labels = torch.argmax(outputs, dim=1)
            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # 各種評価指標の計算
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    tss = tss_score(true_labels, predictions)

    return precision, recall, f1, tss

# モデルの評価
precision, recall, f1, tss = evaluate(model, test_loader)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"TSS: {tss}")
